In [1]:
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import regex as re
from plotly.subplots import make_subplots
import plotly.express as px 
import plotly.graph_objects as go

In [2]:
def pickle_file(obj, path):
    file = open(path, 'ab')
    pickle.dump(obj, file)
    file.close()

def load_pickle(path):
    file = open(path, 'rb')
    obj = pickle.load(file)
    file.close()
    return obj

In [4]:
df = load_pickle(path='./mta_data_cleaned/mta_cord_0_40').reset_index()
# df = df.groupby('STATION').sum().reset_index()
df = df.groupby(['STATION','C/A','UNIT','LINENAME']).sum().reset_index()
df.head()

,STATION,C/A,UNIT,LINENAME,index,ENTRIES,EXITS,PREV_ENTRIES,PREV_EXITS,DAILY_ENTRIES,DAILY_EXITS,OVER_MAX,LINE_DENSITY
0,14 ST-UNION SQ,A033,R170,LNQR456W,100731777607,585713873407,303004125915,5.856928e+11,3.029969e+11,4712737.0,6214879.0,0,1365952.000
1,14 ST-UNION SQ,A034,R170,LNQR456W,125707109297,158719879830,167376792152,1.587250e+11,1.673849e+11,5145094.0,8087369.0,0,1654057.875
2,14 ST-UNION SQ,A035,R170,LNQR456W,63027272870,51016895680,45448237623,5.101894e+10,4.545106e+10,2045809.0,2818508.0,0,608039.625
3,14 ST-UNION SQ,A037,R170,LNQR456W,50288507747,215691894600,53671135387,2.156954e+11,5.367341e+10,3549009.0,2278272.0,0,728410.125
4,14 ST-UNION SQ,R221,R170,456LNQRW,178494196935,411514894035,427157287607,4.115031e+11,4.271550e+11,8062486.0,11208488.0,0,2408871.750


In [3]:
cord_df = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')[['Station ID',
                     'Complex ID', 'Borough', 'GTFS Latitude', 'GTFS Longitude']]
# cord_df.columns = cord_df.columns.str.upper()
# cord_df.columns = cord_df.columns.str.replace(' ', '_')
cord_df.head()


,Station ID,Complex ID,Borough,GTFS Latitude,GTFS Longitude
0,1,1,Q,40.775036,-73.912034
1,2,2,Q,40.770258,-73.917843
2,3,3,Q,40.766779,-73.921479
3,4,4,Q,40.761820,-73.925508
4,5,5,Q,40.756804,-73.929575


In [21]:
zip_borough = pd.read_csv('./mta_data_raw/zip_borough.csv',header=None, names=['zip', 'borough'], index_col=None,usecols=[0,1])
zip_borough.head()

,zip,borough
0,10001,Manhattan
1,10002,Manhattan
2,10003,Manhattan
3,10004,Manhattan
4,10005,Manhattan


In [22]:
station_zip = pd.read_csv('./mta_data_raw/zips.csv')
station_zip.head()

,station,zip_code
0,59 ST,10065
1,5 AV/59 ST,10019
2,57 ST-7 AV,10106
3,49 ST,10020
4,TIMES SQ-42 ST,10018


In [29]:
station_borough = pd.merge(station_zip, zip_borough,left_on='zip_code',right_on='zip', how='left')
station_borough.dropna(subset='borough', inplace=True)
station_borough.drop('zip', axis=1, inplace= True)
station_borough.head()

,station,zip_code,borough
1,5 AV/59 ST,10019,Manhattan
2,5 AV/59 ST,10019,Manhattan
3,57 ST-7 AV,10106,Manhattan
4,57 ST-7 AV,10106,Manhattan
5,49 ST,10020,Manhattan


In [4]:
map_df = pd.read_csv('./mta_data_raw/unit_map.csv')[['remote', 'complex_id']]
map_df.head()

,remote,complex_id
0,R001,635.0
1,R001,635.0
2,R001,635.0
3,R002,628.0
4,R002,628.0


In [ ]:
df = load_pickle(path='./mta_data_cleaned/mta_cord_0_40').reset_index()
# df = df.groupby('STATION').sum().reset_index()
df = df.groupby(['STATION','C/A','UNIT','LINENAME']).sum().reset_index()
df.head()

In [58]:
def merge_and_concat_stations_with_cord(df, map_df, cord_df):

    path = './mta_data_cleaned/'
    columns_of_interest = ['STATION','C/A','UNIT','LINENAME', 'DATE',
        'DAILY_ENTRIES', 'DAILY_EXITS', 'LINE_DENSITY']

    df_list = []
    unmapped = []
    for filename in os.listdir(path):
        if 'stations' not in filename.lower():
            print(f'Skipping:\t{path+filename}', end='\r', flush=True)
            continue
        print(f'Loading:\t{path+filename}', end='\r', flush=True)
        df = (load_pickle(path+filename)[columns_of_interest]
                    .groupby(['UNIT','STATION','C/A','LINENAME','DATE']).sum().reset_index())
        print(f'{path+filename} loaded ({len(df)} rows)', end='\r', flush=True)

        df = pd.merge(df, map_df, how='left', left_on='UNIT', right_on='remote')
        print(f'{path+filename} mapped', end='\r', flush=True)

        df = pd.merge(df, cord_df, how='left', left_on='complex_id', right_on='Complex ID')
        print(f'{path+filename} merged with coordinates', end='\r', flush=True)

        leftover_mask = df['complex_id'].isna()
        unmapped.append(df[leftover_mask])
        print(f'{path+filename} done. \t\tUnmapped Units:\t{len(leftover_mask)}', flush=True)
        df_list.append(df)
    return df_list

df_list = merge_and_concat_stations_with_cord(df, map_df, cord_df)

df = pd.concat(df_list)
print('Data Concactenated', end='\r', flush=True)

df.columns = df.columns.str.upper()
df.columns = df.columns.str.replace(' ', '_')
# df.sort_values('DAILY_ENTRIES',ascending=False)
pickle_file(df, './mta_data_cleaned/mta_with_cordinates')
print(f'Saved:\t./mta_data_cleaned/mta_with_cordinates')

./mta_data_cleaned/mta_stations_360_400 done. 		Unmapped Units:	27555
./mta_data_cleaned/mta_stations_120_160 done. 		Unmapped Units:	341489
./mta_data_cleaned/mta_stations_200_240 done. 		Unmapped Units:	129770
./mta_data_cleaned/mta_stations_40_80 done. 		Unmapped Units:	189716
./mta_data_cleaned/mta_stations_160_200 done. 		Unmapped Units:	163113
./mta_data_cleaned/mta_stations_240_280 done. 		Unmapped Units:	163085
./mta_data_cleaned/mta_stations_320_360 done. 		Unmapped Units:	69117
./mta_data_cleaned/mta_stations_0_40 done. 		Unmapped Units:	721443
./mta_data_cleaned/mta_stations_280_320 done. 		Unmapped Units:	126621
./mta_data_cleaned/mta_stations_80_120 done. 		Unmapped Units:	69419
Saved:	./mta_data_cleaned/mta_with_cordinates


In [89]:
def merge_with_covid_boroughs():
    df = load_pickle('./mta_data_cleaned/mta_with_cordinates')
    df = df.groupby(['BOROUGH','DATE']).sum().reset_index()

    for column in ['DAILY_ENTRIES','DAILY_EXITS', 'LINE_DENSITY']:
        for borough in df.BOROUGH.unique():
            df['MASK'] = df.BOROUGH==borough
            df[f'{borough}_{column}'] = df.MASK * df[column]
    df.drop('MASK', axis = 1, inplace=True)
    df = df[['DATE', 'DAILY_ENTRIES', 'DAILY_EXITS', 'LINE_DENSITY',
        'Bk_DAILY_ENTRIES', 'Bx_DAILY_ENTRIES',
        'M_DAILY_ENTRIES', 'Q_DAILY_ENTRIES', 'SI_DAILY_ENTRIES',
        'Bk_DAILY_EXITS', 'Bx_DAILY_EXITS', 'M_DAILY_EXITS', 'Q_DAILY_EXITS',
        'SI_DAILY_EXITS', 'Bk_LINE_DENSITY', 'Bx_LINE_DENSITY',
        'M_LINE_DENSITY', 'Q_LINE_DENSITY', 'SI_LINE_DENSITY']]
    df = df.groupby(['DATE']).sum().reset_index()

    df_covid = pd.read_csv('./covid_data_raw/COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv')
    df_covid.DATE_OF_INTEREST = pd.to_datetime(df_covid.DATE_OF_INTEREST)

    # Retrieved list of columns from ~ df_covid.columns
    df_covid = df_covid[['DATE_OF_INTEREST', 'CASE_COUNT', 'probable_case_count',
        'HOSPITALIZED_COUNT', 'DEATH_COUNT', 'DEATH_COUNT_PROBABLE',
        'CASE_COUNT_7DAY_AVG', 'all_case_count_7day_avg', 'HOSP_COUNT_7DAY_AVG',
        'DEATH_COUNT_7DAY_AVG', 'all_death_count_7day_avg']]
    
    df = pd.merge(df, df_covid, left_on='DATE', right_on='DATE_OF_INTEREST')

    return df

df = merge_with_covid_boroughs()
df.sample(15)
pickle_file(df, './data_merged/mta_broughs_cases')
print(f'Saved:\t./data_merged/mta_broughs_cases')

Saved:	./data_merged/mta_broughs_cases


In [34]:
def convert_data_by_stations():
    df = load_pickle('./data_merged/mta_broughs_cases')
    stations = load_pickle('./mta_data_cleaned/mta_with_cordinates')
    stations =  stations[['STATION', 'BOROUGH']].drop_duplicates().groupby('BOROUGH').count()
    for column in df.columns:
        # print(column.split('_')[0])
        if column.split('_')[0] in stations.index:
            df[column] = df[column] / stations.loc[column.split('_')[0]].STATION
            print(f'reduced {column}', end='\r', flush=True)
    pickle_file(df, './data_merged/mta_broughs_cases_by_num_stations')
    print(f'Saved:\t./data_merged/mta_broughs_cases_by_num_stations')
    return df

df = convert_data_by_stations()
df.head()

Saved:	./data_merged/mta_broughs_cases_by_num_stations


,DATE,DAILY_ENTRIES,DAILY_EXITS,LINE_DENSITY,Bk_DAILY_ENTRIES,Bx_DAILY_ENTRIES,M_DAILY_ENTRIES,Q_DAILY_ENTRIES,SI_DAILY_ENTRIES,Bk_DAILY_EXITS,...,CASE_COUNT,probable_case_count,HOSPITALIZED_COUNT,DEATH_COUNT,DEATH_COUNT_PROBABLE,CASE_COUNT_7DAY_AVG,all_case_count_7day_avg,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,all_death_count_7day_avg
0,2020-02-29,1.098717e+07,1.014809e+07,6.651748e+06,11795.214815,7751.344262,82691.834725,10434.012821,2166.0,9637.029630,...,1,0,1,0,0,0,0,0,0,0
1,2020-03-01,8.332771e+06,7.565224e+06,5.075832e+06,8907.155556,5860.377049,62457.806122,8317.410256,1599.5,7720.259259,...,0,0,1,0,0,0,0,0,0,0
2,2020-03-02,2.107111e+07,1.813919e+07,1.258845e+07,21323.555556,14387.770492,160325.607666,20200.858974,13598.0,16085.748148,...,0,0,2,0,0,0,0,0,0,0
3,2020-03-03,2.175668e+07,1.880443e+07,1.302943e+07,21778.162963,14470.156617,165950.454605,21060.843979,14031.0,16377.622222,...,1,0,7,0,0,0,0,0,0,0
4,2020-03-04,2.183703e+07,1.891770e+07,1.302833e+07,22010.422222,14631.508197,166260.993154,21180.846154,13710.5,16771.437037,...,5,0,2,0,0,0,0,0,0,0


In [30]:
df = load_pickle('./mta_data_cleaned/mta_with_cordinates')
stations =  df[['STATION', 'BOROUGH']].drop_duplicates().groupby('BOROUGH').count()
stations.loc['Bk'].STATION
# df.drop_duplicates

135

In [78]:
df = df.groupby(['BOROUGH','DATE']).sum().reset_index()

for column in ['DAILY_ENTRIES','DAILY_EXITS', 'LINE_DENSITY']:
    for borough in df.BOROUGH.unique():
        df['MASK'] = df.BOROUGH==borough
        df[f'{borough}_{column}'] = df.MASK * df[column]
df.drop('MASK', axis = 1, inplace=True)
df = df['DATE', 'DAILY_ENTRIES', 'DAILY_EXITS', 'LINE_DENSITY',
       'Bk_DAILY_ENTRIES', 'Bx_DAILY_ENTRIES',
       'M_DAILY_ENTRIES', 'Q_DAILY_ENTRIES', 'SI_DAILY_ENTRIES',
       'Bk_DAILY_EXITS', 'Bx_DAILY_EXITS', 'M_DAILY_EXITS', 'Q_DAILY_EXITS',
       'SI_DAILY_EXITS', 'Bk_LINE_DENSITY', 'Bx_LINE_DENSITY',
       'M_LINE_DENSITY', 'Q_LINE_DENSITY', 'SI_LINE_DENSITY']
df = df.groupby(['DATE']).sum().reset_index()
# df['sum_values_A'] = df.groupby('A')['values'].transform(np.sum)

,DAILY_ENTRIES,DAILY_EXITS,LINE_DENSITY,COMPLEX_ID,STATION_ID,COMPLEX_ID,GTFS_LATITUDE,GTFS_LONGITUDE,Bk_DAILY_ENTRIES,Bx_DAILY_ENTRIES,...,Bk_DAILY_EXITS,Bx_DAILY_EXITS,M_DAILY_EXITS,Q_DAILY_EXITS,SI_DAILY_EXITS,Bk_LINE_DENSITY,Bx_LINE_DENSITY,M_LINE_DENSITY,Q_LINE_DENSITY,SI_LINE_DENSITY
DATE,,,,,,,,,,,,,,,,,,,,,
2019-12-28,1.327988e+07,1.203084e+07,7.522520e+06,1027373.0,566732.0,1027373.0,93973.487875,-170644.944758,1.423923e+06,492256.0,...,1.219665e+06,354218.0,9.682229e+06,774730.0,0.0,1.229529e+06,503121.666667,4.828877e+06,9.514569e+05,9535.0
2019-12-29,1.052463e+07,9.574988e+06,6.028927e+06,1027373.0,566732.0,1027373.0,93973.487875,-170644.944758,1.162094e+06,372894.0,...,1.010592e+06,281241.0,7.673138e+06,610017.0,0.0,1.022308e+06,388040.333333,3.875677e+06,7.372029e+05,5699.0
2019-12-30,1.820172e+07,1.586223e+07,1.052618e+07,1027373.0,566732.0,1027373.0,93973.487875,-170644.944758,2.083494e+06,660139.0,...,1.642329e+06,410755.0,1.278780e+07,1021346.0,0.0,1.706702e+06,648587.666667,6.835060e+06,1.315776e+06,20058.0
2019-12-31,1.611535e+07,1.511402e+07,9.990003e+06,1027373.0,566732.0,1027373.0,93973.487875,-170644.944758,2.178321e+06,657436.0,...,1.766040e+06,417756.0,1.194257e+07,987650.0,0.0,1.789648e+06,648261.666667,6.250240e+06,1.283655e+06,18199.0
2020-01-01,9.080155e+06,8.349830e+06,5.344640e+06,1027373.0,566732.0,1027373.0,93973.487875,-170644.944758,1.053358e+06,316308.0,...,9.319430e+05,241348.0,6.646347e+06,530192.0,0.0,9.304385e+05,337026.333333,3.426951e+06,6.446729e+05,5551.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-25,8.228198e+06,1.101961e+07,6.510174e+06,1047444.0,577670.0,1047444.0,95603.759229,-173604.115612,1.257432e+06,416974.0,...,1.548209e+06,398564.0,8.202566e+06,870269.0,0.0,1.253935e+06,511271.666667,3.773867e+06,9.600399e+05,11060.0
2022-04-26,8.834816e+06,1.202572e+07,7.005805e+06,1047444.0,577670.0,1047444.0,95603.759229,-173604.115612,1.275222e+06,469436.0,...,1.526971e+06,453852.0,9.140605e+06,904290.0,0.0,1.291595e+06,553723.500000,4.156737e+06,9.916179e+05,12132.0
2022-04-27,9.279071e+06,1.269535e+07,7.323588e+06,1047444.0,577670.0,1047444.0,95603.759229,-173604.115612,1.319792e+06,498194.0,...,1.592432e+06,483871.0,9.691989e+06,927059.0,0.0,1.341469e+06,579854.166667,4.367375e+06,1.023079e+06,11811.0
